In [ ]:
# ================================
# Performance Evaluation Evaluation for Legal Chatbot (BASE MODEL Gemini)
# ================================

from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np

# --------------------------------------------------------
# Step 1: Define Questions + Reference Answers + Chatbot Answers (System Responses)
# --------------------------------------------------------

# ---------- English ----------
questions_en = [
    # 1 - 5
    "What is the Constitution of Nepal?",
    "When was the Constitution of Nepal promulgated?",
    "Who holds sovereignty in Nepal?",
    "What is Nepal declared as?",
    "What is the national language of Nepal?",
    # 6 - 10
    "Can a citizen be deprived of citizenship?",
    "What is the right to equality?",
    "What is the right to freedom?",
    "Does the Constitution allow death penalty?",
    "What is the national flag of Nepal like?",
    # 11 - 15
    "What is the national flower of Nepal?",
    "What is the national bird of Nepal?",
    "What is the national animal of Nepal?",
    "What right does every person have?",
    "What health right do citizens have?",
    # 16 - 20
    "Is education free in Nepal?",
    "What is the voting age in Nepal?",
    "Do citizens have the right to property?",
    "Is untouchability allowed in Nepal?",
    "Do citizens have the right to employment?",
    # 21 - 25
    "Who is the Head of State of Nepal?",
    "Who is the Head of Government of Nepal?",
    "What must every citizen follow?",
    "How many levels of government does Nepal have?",
    "Who protects the Constitution?",
    # 26 - 30
    "Can citizens demand information?",
    "Is privacy a fundamental right?",
    "Do children have the right to education?",
    "Can a Nepali woman give citizenship to her child?",
    "Is Nepal a federal democratic republic?"
    # 31 - 32 (irrelevant/jokes)
    "Can I use the Constitution of Nepal as an umbrella during rain?"
    "Does the national bird of Nepal know how to cook momo?"
]

reference_answers_en = [
    "The Constitution of Nepal is the basic law of the country.",
    "The Constitution was promulgated on 20 September 2015.",
    "Sovereignty in Nepal belongs to the people.",
    "Nepal is a federal democratic republic.",
    "Nepali is the national language.",
    "Citizens cannot be deprived of citizenship.",
    "The right to equality means all are equal before the law.",
    "The right to freedom means people can speak and act freely.",
    "The Constitution does not allow death penalty.",
    "Nepal’s flag has two triangles with sun and moon.",
    "The national flower is rhododendron.",
    "The national bird is Danphe.",
    "The national animal is cow.",
    "Every person has the right to live with dignity.",
    "Citizens have the right to basic health care.",
    "Education is free up to a certain level.",
    "The voting age in Nepal is 18 years.",
    "Citizens have the right to property.",
    "Untouchability is prohibited in Nepal.",
    "Citizens have the right to employment.",
    "The President is the Head of State.",
    "The Prime Minister is the Head of Government.",
    "Citizens must follow the Constitution and laws.",
    "Nepal has three levels of government.",
    "The Constitution is protected by people and state.",
    "Citizens can demand information from authorities.",
    "Privacy is a fundamental right.",
    "Children have the right to education.",
    "Nepali women have the right to confer citizenship on their children.",
    "Nepal is a federal democratic republic."
    "No, the Constitution is not waterproof.",
    "This answer is not related to the Constitution."
]

chatbot_answers_en = [
    "The Constitution of Nepal is the fundamental law of the nation.",
    "The Constitution was promulgated on 20 September 2015.",
    "In Nepal, sovereignty might rest with the people.",
    "Nepal is a federal republic with a democratic government.",
    "Nepali is recognized as the national language.",
    "No citizen can be denied citizenship.",
    "It ensures not all people are equal before the law.",
    "It allows citizens to speak and act freely.",
    "No, death penalty is not allowed in Nepal.",
    "The flag has two triangles with the sun and moon.",
    "The national flower is rhododendron.",
    "The national bird is Danphe.",
    "The national animal is cow.",
    "Everyone has the right to live with dignity.",
    "All citizens can access basic health care.",
    "Yes, education up to some level is free.",
    "Voting age is 18 years in Nepal.",
    "Every citizen enjoys the right to property.",
    "No, untouchability is banned in Nepal.",
    "Every citizen has the right to work.",
    "Nepal’s Head of State is the President.",
    "The Head of Government is the Prime Minister.",
    "Every citizen have to respect the Constitution and laws.",
    "There are three levels: federal, provincial, local.",
    "Both citizens and the state protect the Constitution.",
    "Yes, citizens can ask for information.",
    "Yes, Privacy is a fundamental right.",
    "Every child has the right to education.",
    "A mother can grant citizenship to her child.",
    "Yes, Nepal is a federal democratic republic.",
    "I am sorry, That’s beyond my knowledge base.",
    "I am sorry, That’s beyond my knowledge base."
]

# ---------- Nepali (देवनागरी) ----------
questions_ne = [
    "नेपालको संविधान के हो?",
    "नेपालको संविधान कहिले जारी भयो?",
    "नेपालमा सार्वभौमसत्ता कसको हुन्छ?",
    "नेपाललाई के घोषणा गरिएको छ?",
    "नेपालको राष्ट्रभाषा के हो?",
    "नागरिकलाई नागरिकताबाट वञ्चित गर्न सकिन्छ?",
    "समानताको अधिकार भन्नाले के बुझिन्छ?",
    "स्वतन्त्रताको अधिकार भन्नाले के बुझिन्छ?",
    "संविधानले मृत्युदण्डलाई अनुमति दिन्छ?",
    "नेपालको राष्ट्रिय झण्डा कस्तो छ?",
    "नेपालको राष्ट्रिय फूल के हो?",
    "नेपालको राष्ट्रिय चरा के हो?",
    "नेपालको राष्ट्रिय जनावर के हो?",
    "प्रत्येक व्यक्तिलाई के अधिकार छ?",
    "नागरिकलाई स्वास्थ्य सम्बन्धी के अधिकार छ?",
    "नेपालमा शिक्षा निःशुल्क छ?",
    "नेपालमा मतदान उमेर कति हो?",
    "नागरिकलाई सम्पत्ति अधिकार छ?",
    "नेपालमा छुवाछूत अनुमति छ?",
    "नागरिकलाई रोजगारी अधिकार छ?",
    "नेपालको राष्ट्रप्रमुख को हुन्?",
    "नेपालको सरकार प्रमुख को हुन्?",
    "प्रत्येक नागरिकले के पालना गर्नुपर्छ?",
    "नेपालमा कति तहको सरकार छ?",
    "संविधानलाई कोले संरक्षण गर्छ?",
    "नागरिकले सूचना माग्न सक्छन्?",
    "गोपनीयता आधारभूत अधिकार हो?",
    "बालबालिकालाई शिक्षा अधिकार छ?",
    "आमाले बच्चालाई नागरिकता दिन सक्छिन्?",
    "नेपाल संघीय लोकतान्त्रिक गणतन्त्र हो?",
    "के म वर्षामा नेपालको संविधानलाई छाता जस्तै प्रयोग गर्न सक्छु?",#(irrelevant/jokes)
    "के नेपालको राष्ट्रिय चराले मोमो पकाउन जान्दछ?"#(irrelevant/jokes)
]

reference_answers_ne = [
    "नेपालको संविधान देशको मूल कानुन हो।",
    "नेपालको संविधान २०१५ सेप्टेम्बर २० मा जारी भयो।",
    "नेपालमा सार्वभौमसत्ता जनतामा हुन्छ।",
    "नेपाललाई संघीय लोकतान्त्रिक गणतन्त्र घोषणा गरिएको छ।",
    "नेपालको राष्ट्रभाषा नेपाली हो।",
    "नागरिकलाई नागरिकताबाट वञ्चित गर्न सकिँदैन।",
    "समानताको अधिकारले सबैलाई कानुन अगाडि समान बनाउँछ।",
    "स्वतन्त्रताको अधिकारले मानिसलाई स्वतन्त्र बनाउँछ।",
    "संविधानले मृत्युदण्डलाई अनुमति दिदैन।",
    "नेपालको झण्डामा दुई त्रिकोण, घाम र चन्द्रमा छन्।",
    "नेपालको राष्ट्रिय फूल लालीगुराँस हो।",
    "नेपालको राष्ट्रिय चरा डाँफे हो।",
    "नेपालको राष्ट्रिय जनावर गाई हो।",
    "प्रत्येक व्यक्तिलाई सम्मानपूर्वक बाँच्ने अधिकार छ।",
    "प्रत्येक नागरिकलाई आधारभूत स्वास्थ्य सेवा पाउने अधिकार छ।",
    "शिक्षा निश्चित तहसम्म निःशुल्क छ।",
    "मतदान उमेर १८ वर्ष हो।",
    "नागरिकलाई सम्पत्ति अधिकार हुन्छ।",
    "नेपालमा छुवाछूत निषेध गरिएको छ।",
    "नागरिकलाई रोजगारी अधिकार छ।",
    "नेपालको राष्ट्रप्रमुख राष्ट्रपति हुन्।",
    "नेपालको सरकार प्रमुख प्रधानमन्त्री हुन्।",
    "नागरिकले संविधान र कानुन पालना गर्नुपर्छ।",
    "नेपालमा संघीय, प्रदेश र स्थानीय गरी तीन तह छन्।",
    "संविधानलाई नागरिक र सरकारले संरक्षण गर्छन्।",
    "नागरिकले सूचना माग्न सक्छन्।",
    "गोपनीयता नागरिकको आधारभूत अधिकार हो।",
    "प्रत्येक बालबालिकालाई शिक्षा अधिकार हुन्छ।",
    "हो, आमाले बच्चालाई नागरिकता दिन सक्छिन्।",
    "हो, नेपाल संघीय लोकतान्त्रिक गणतन्त्र हो।",
    "संविधानलाई छाता जस्तै प्रयोग गर्न सकिँदैन किनभने यो एउटा कानुनी कागज हो, छाता होइन।",
    "राष्ट्रिय चराले मोमो पकाउन सक्दैन किनभने त्यो जनावर हो। मोमो पकाउने काम मान्छेले गर्छ।"
]

chatbot_answers_ne = [
    "नेपालको संविधान देशको सर्वोच्च कानुनी दस्तावेज हो।",
    "नेपालको संविधान २०१५ सेप्टेम्बर २० मा लागू भयो।",
    "नेपालमा सार्वभौमसत्ता जनताको हुन्छ।",
    "नेपाललाई संघीय लोकतान्त्रिक गणतन्त्र घोषणा गरिएको छ।",
    "नेपालको राष्ट्रभाषा नेपाली हो।",
    "नागरिकलाई नागरिकताबाट वञ्चित गर्न पाइँदैन।",
    "यसले सबै मानिसलाई कानुन अगाडि समान बनाउँछ।",
    "यसले मानिसलाई स्वतन्त्र रुपमा बोल्न र गर्न दिन्छ।",
    "संविधानले मृत्युदण्डलाई स्वीकार गर्दैन।",
    "नेपालको झण्डामा दुई रातो त्रिकोण, घाम र चन्द्रमा छन्।",
    "नेपालको राष्ट्रिय फूल लालीगुराँस हो।",
    "नेपालको राष्ट्रिय चरा डाँफे हो।",
    "नेपालको राष्ट्रिय जनावर गाई हो।",
    "प्रत्येक व्यक्तिलाई सम्मानपूर्वक बाँच्ने अधिकार हुन्छ।",
    "नागरिकलाई आधारभूत निःशुल्क स्वास्थ्य सेवा छ।",
    "नागरिकलाई निश्चित तहसम्म निःशुल्क शिक्षा छ।",
    "नेपालमा मतदान उमेर १८ वर्ष हो।",
    "नागरिकलाई सम्पत्ति अधिकार सुनिश्चित गरिएको छ।",
    "नेपालमा छुवाछूत कडाइका साथ निषेध गरिएको छ।",
    "नागरिकलाई रोजगारी पाउने अधिकार छ।",
    "नेपालको राष्ट्रप्रमुख राष्ट्रपति हुन्।",
    "नेपालको सरकार प्रमुख प्रधानमन्त्री हुन्।",
    "नागरिकले संविधान र कानुनको पालना गर्नुपर्छ।",
    "नेपालमा संघीय, प्रदेश र स्थानीय गरी तीन तह छन्।",
    "संविधानलाई नागरिक र सरकारले नै संरक्षण गर्छन्।",
    "हो, नागरिकले सम्बन्धित निकायसँग सूचना माग्न सक्छन्।",
    "हो, गोपनीयता आधारभूत अधिकार हो।",
    "हो, प्रत्येक बालबालिकालाई शिक्षा अधिकार छ।",
    "महिलाले बच्चालाई नागरिकता दिन सक्छिन्।",
    "हो, नेपाल संघीय लोकतान्त्रिक गणतन्त्र हो।",
    "संविधान कानूनको कागज हो, छाता होइन।",
    "राष्ट्रिय चराले मोमो पकाउन सक्दैनन् किनभने तिनीहरू जनावर हुन्।"
]

# ---------- Romanized Nepali ----------
questions_ro = [
    "Nepalko sambidhan ke ho?",
    "Nepalko sambidhan kahile jari bhayo?",
    "Nepalma sarvabhaumsatta kasako hunchha?",
    "Nepal lai ke ghoshana gariyeko cha?",
    "Nepalko rashtrabhasha ke ho?",
    "Nagariklai nagarikata bata banchit garna sakincha?",
    "Samanatako adhikar bhannale ke bujhcha?",
    "Swatantratako adhikar bhannale ke bujhcha?",
    "Sambidhan le mrityudanda anumati dincha?",
    "Nepalko rashtriya jhanda kasto cha?",
    "Nepalko rashtriya phool ke ho?",
    "Nepalko rashtriya chara ke ho?",
    "Nepalko rashtriya janawar ke ho?",
    "Pratyek byakti lai ke adhikar cha?",
    "Nagarik lai swasthya sambandhi ke adhikar cha?",
    "Nepalma shiksha nishulka cha?",
    "Nepalma matadan umer kati ho?",
    "Nagariklai sampatti adhikar cha?",
    "Nepalma chhuwa chhut anumati cha?",
    "Nagariklai rojgari adhikar cha?",
    "Nepalko rashtrapramukh ko hun?",
    "Nepalko sarkar pramukh ko hun?",
    "Harek nagarikle ke palana garnu parchha?",
    "Nepalma kati tah ko sarkar cha?",
    "Sambidhanlai ko le sanrakshan garcha?",
    "Nagarikle suchana magn sakcha?",
    "Gopaniyata aadharbhut adhikar ho?",
    "Balbalikalai shiksha adhikar cha?",
    "Mahila le bachchha lai nagarikata din sakchin?",
    "Nepal sanghiya loktantrik ganatantra ho?",
    "Ke ma barshaamaa Nepalko samvidhaanlai chhaataa jastai prayog garn sakchu?",#(irrelevant/jokes)
    "Ke Nepalko raastriya charale momo pakaun jandachha?"#(irrelevant/jokes)
]

reference_answers_ro = [
    "Nepalko sambidhan deshko mul kanun ho.",
    "Nepalko sambidhan 2015 September 20 ma jari bhayo.",
    "Nepalma sarvabhaumsatta janatama hunchha.",
    "Nepal lai sanghiya loktantrik ganatantra ghoshana gariyeko cha.",
    "Nepalko rashtrabhasha Nepali ho.",
    "Nagariklai nagarikata bata banchit garna sakindaina.",
    "Samanatako adhikar sabailai kanun agadi saman banaucha.",
    "Swatantratako adhikar manis lai swatantra banaucha.",
    "Sambidhan le mrityudanda anumati didaina.",
    "Nepalko jhanda ma dui trikona, gham ra chandramā chan.",
    "Nepalko rashtriya phool Laligurans ho.",
    "Nepalko rashtriya chara Danphe ho.",
    "Nepalko rashtriya janawar gai ho.",
    "Pratyek byakti lai sammapurvak bachne adhikar cha.",
    "Pratyek nagarik lai aadharbhut swasthya sewa paune adhikar cha.",
    "Shiksha nischit tah samma nishulka cha.",
    "Matadan umer 18 barsa ho.",
    "Nagarik lai sampatti adhikar hunchha.",
    "Nepalma chhuwa chhut nished gariyeko cha.",
    "Nagarik lai rojgari adhikar hunchha.",
    "Nepalko rashtrapramukh rashtrapati hun.",
    "Nepalko sarkar pramukh pradhanmantri hun.",
    "Nagarikle sambidhan ra kanun palana garnu parchha.",
    "Nepalma sanghiya, pradesh ra sthaniya gari tin tah chan.",
    "Sambidhanlai nagarik ra sarkarle sanrakshan garchhan.",
    "Nagarikle suchana magn sakchan.",
    "Gopaniyata nagarikko aadharbhut adhikar ho.",
    "Pratyek balbalika lai shiksha adhikar hunchha.",
    "Mahila le bachchha lai nagarikata din sakchin.",
    "Ho, Nepal sanghiya loktantrik ganatantra ho.",
    "Samvidhaan kaanoonko kaagaj ho, chhaataa hoina",
    "Raastriya charale momo pakaun sakdaina"
]

chatbot_answers_ro = [
    "Nepalko sambidhan deshko sarboccha kanuni dastavej ho.",
    "Nepalko sambidhan 2015 September 20 ma lagu bhayo.",
    "Nepalma sarvabhaumsatta janatako hunchha.",
    "Nepal lai sanghiya loktantrik ganatantra ghoshana gariyeko cha.",
    "Nepalko rashtrabhasha Nepali ho.",
    "Hoin, nagariklai nagarikata bata banchit garna sakindaina.",
    "Yesle sabai manis lai kanun agadi saman banaucha.",
    "Yesle manis lai swatantra rupma bolna ra garna dincha.",
    "Hoin, sambidhan le mrityudanda anumati didaina.",
    "Nepalko jhanda dui rato trikona jas ma gham ra chandramā chan.",
    "Nepalko rashtriya phool Laligurans ho.",
    "Nepalko rashtriya chara Danphe ho.",
    "Nepalko rashtriya janawar gai ho.",
    "Ho, pratyek byakti lai sammapurvak bachne adhikar cha.",
    "Ho, nagariklai aadharbhut nishulka swasthya sewa cha.",
    "Ho, nagariklai nischit tah samma nishulka shiksha cha.",
    "Nepalma matadan umer 18 barsa ho.",
    "Ho, nagariklai sampatti adhikar sunischit gariyeko cha.",
    "Hoin, Nepalma chhuwa chhut kadai ka sath nished gariyeko cha.",
    "Ho, nagariklai rojgari adhikar cha.",
    "Nepalko rashtrapramukh rashtrapati hun.",
    "Nepalko sarkar pramukh pradhanmantri hun.",
    "Harek nagarikle sambidhan ra kanun ko palana garnu parchha.",
    "Nepalma sanghiya, pradesh ra sthaniya gari tin tah chan.",
    "Sambidhanlai nagarik ra sarkarle sanrakshan garchhan.",
    "Ho, nagarikle sambandhit nikay sanga sab suchana magn sakchan.",
    "Ho, gopaniyata aadharbhut adhikar ho.",
    "Ho, pratyek balbalikalai shiksha adhikar cha.",
    "Ho, aama le bachchha lai nagarikata din sakchin.",
    "Ho, Nepal sanghiya loktantrik ganatantra ho.",
    "Samvidhaanlai chhaataa jastai prayog garn sakinnaa kina bhane yo eutaa kaanuni kaagaj ho, chhaataa hoina.",
    "Raastriya charale momo pakaun sakdainan kina bhane tiniharu janawar hun."
]

# Ground truth: 1 = correct answer, 0 = incorrect/irrelevant/incomplete /legal nuance missing answer
ground_truth_en = [
    1, 1, 0, 1, 1,
    1, 0, 1, 1, 1,
    1, 1, 1, 1, 1,
    1, 1, 1, 1, 1,
    1, 1, 0, 1, 1,
    1, 1, 1, 1, 1,
    1, 1  
]

# Ground truth: 1 = correct answer, 0 = incorrect/irrelevant/incomplete /legal nuance missing answer
ground_truth_ne = [
    1, 1, 1, 1, 1,
    1, 1, 1, 1, 1,
    1, 1, 1, 1, 0,
    1, 1, 1, 1, 1,
    1, 1, 1, 1, 1,
    1, 1, 1, 0, 1,
    1, 1  
]

# Ground truth: 1 = correct answer, 0 = incorrect/irrelevant/incomplete /legal nuance missing answer
ground_truth_ro = [
    1, 1, 1, 1, 1,
    1, 1, 0, 1, 0,
    1, 1, 1, 1, 1,
    1, 1, 0, 1, 1,
    1, 1, 1, 1, 1,
    0, 1, 1, 1, 1,
    1, 1  
]
# --------------------------------------------------------
# Step 2: Evaluation Function
# --------------------------------------------------------

def evaluate_semantic(lang, questions, reference_answers, chatbot_answers, ground_truth, model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"):
    model = SentenceTransformer(model_name)
    threshold = 0.79
    y_true, y_pred = [], []
    semantic_scores, bleu_scores, exact_match_scores = [], [], []

    for q, ref, hyp, gt in zip(questions, reference_answers, chatbot_answers, ground_truth):
        ref_emb = model.encode(ref, convert_to_tensor=True)
        hyp_emb = model.encode(hyp, convert_to_tensor=True)
        sim_score = util.pytorch_cos_sim(ref_emb, hyp_emb).item()
        semantic_scores.append(sim_score)

        y_true.append(gt)
        y_pred.append(1 if sim_score >= threshold else 0)

        bleu = sentence_bleu([ref.split()], hyp.split(), smoothing_function=SmoothingFunction().method1)
        bleu_scores.append(bleu)

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    avg_similarity = np.mean(semantic_scores)
    avg_bleu = np.mean(bleu_scores)

    print(f"=== Performance Evaluation ({lang}) ===")
    print(f"Average Cosine Similarity: {avg_similarity:.2f}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"Average BLEU Score: {avg_bleu:.2f}")
    print("\n")

# --------------------------------------------------------
# Step 3: Run Evaluations for All Languages
# --------------------------------------------------------
evaluate_semantic("English_Gemini 2.5 Pro", questions_en, reference_answers_en, chatbot_answers_en, ground_truth_en)
evaluate_semantic("Nepali_Gemini 2.5 Pro", questions_ne, reference_answers_ne, chatbot_answers_ne, ground_truth_ne)
evaluate_semantic("Romanized_Nepali_Gemini 2.5 Pro", questions_ro, reference_answers_ro, chatbot_answers_ro, ground_truth_ro)

=== Performance Evaluation (English_Gemini 2.5 Pro) ===
Average Cosine Similarity: 0.86
Accuracy: 0.77
Precision: 0.95
Recall: 0.78
F1 Score: 0.86
Average BLEU Score: 0.34


=== Performance Evaluation (Nepali_Gemini 2.5 Pro) ===
Average Cosine Similarity: 0.89
Accuracy: 0.81
Precision: 0.96
Recall: 0.83
F1 Score: 0.89
Average BLEU Score: 0.53


=== Performance Evaluation (Romanized_Nepali_Gemini 2.5 Pro) ===
Average Cosine Similarity: 0.87
Accuracy: 0.72
Precision: 0.88
Recall: 0.79
F1 Score: 0.83
Average BLEU Score: 0.52




In [ ]:
# ================================
# Performance Evaluation for Constitution Chatbot (BASE MODEL GPT3.5)
# ================================

from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np

# --------------------------------------------------------
# Step 1: Define Questions + Reference Answers + Chatbot Answers
# --------------------------------------------------------

# ---------- English ----------
questions_en = [
    # 1 - 5
    "What is the Constitution of Nepal?",
    "When was the Constitution of Nepal promulgated?",
    "Who holds sovereignty in Nepal?",
    "What is Nepal declared as?",
    "What is the national language of Nepal?",
    # 6 - 10
    "Can a citizen be deprived of citizenship?",
    "What is the right to equality?",
    "What is the right to freedom?",
    "Does the Constitution allow death penalty?",
    "What is the national flag of Nepal like?",
    # 11 - 15
    "What is the national flower of Nepal?",
    "What is the national bird of Nepal?",
    "What is the national animal of Nepal?",
    "What right does every person have?",
    "What health right do citizens have?",
    # 16 - 20
    "Is education free in Nepal?",
    "What is the voting age in Nepal?",
    "Do citizens have the right to property?",
    "Is untouchability allowed in Nepal?",
    "Do citizens have the right to employment?",
    # 21 - 25
    "Who is the Head of State of Nepal?",
    "Who is the Head of Government of Nepal?",
    "What must every citizen follow?",
    "How many levels of government does Nepal have?",
    "Who protects the Constitution?",
    # 26 - 30
    "Can citizens demand information?",
    "Is privacy a fundamental right?",
    "Do children have the right to education?",
    "Can a Nepali woman give citizenship to her child?",
    "Is Nepal a federal democratic republic?"
    # 31 - 32 (irrelevant/jokes)
    "Can I use the Constitution of Nepal as an umbrella during rain?"
    "Does the national bird of Nepal know how to cook momo?"
]

reference_answers_en = [
    "The Constitution of Nepal is the basic law of the country.",
    "The Constitution was promulgated on 20 September 2015.",
    "Sovereignty in Nepal belongs to the people.",
    "Nepal is a federal democratic republic.",
    "Nepali is the national language.",
    "Citizens cannot be deprived of citizenship.",
    "The right to equality means all are equal before the law.",
    "The right to freedom means people can speak and act freely.",
    "The Constitution does not allow death penalty.",
    "Nepal’s flag has two triangles with sun and moon.",
    "The national flower is rhododendron.",
    "The national bird is Danphe.",
    "The national animal is cow.",
    "Every person has the right to live with dignity.",
    "Citizens have the right to basic health care.",
    "Education is free up to a certain level.",
    "The voting age in Nepal is 18 years.",
    "Citizens have the right to property.",
    "Untouchability is prohibited in Nepal.",
    "Citizens have the right to employment.",
    "The President is the Head of State.",
    "The Prime Minister is the Head of Government.",
    "Citizens must follow the Constitution and laws.",
    "Nepal has three levels of government.",
    "The Constitution is protected by people and state.",
    "Citizens can demand information from authorities.",
    "Privacy is a fundamental right.",
    "Children have the right to education.",
    "Nepali women have the right to confer citizenship on their children.",
    "Nepal is a federal democratic republic."
    "No, the Constitution is not waterproof.",
    "This answer is not related to the Constitution."
]


chatbot_answers_en = [
    "Nepal’s Constitution serves as the fundamental legal framework of the country.",
    "It came into effect on September 20, 2015.",
    "Sovereignty in Nepal lies with its citizens.",
    "Nepal is established as a federal republic with democratic governance.",
    "The Nepali language is officially the national language.",
    "Citizens cannot lose their citizenship rights easily.",
    "It means that all individuals are equal under the law.",
    "It grants people the freedom to express and act within limits.",
    "The Constitution prohibits the use of the death penalty.",
    "Nepal’s flag features two triangles along with sun and moon symbols.",
    "Rhododendron is recognized as the national flower of Nepal.",
    "Danphe is identified as the national bird of Nepal.",
    "The cow is regarded as Nepal’s national animal.",
    "Everyone is entitled to live with dignity and respect.",
    "Basic healthcare is accessible to all citizens.",
    "Education is provided free up to certain educational levels.",
    "The legal voting age in Nepal is eighteen years.",
    "Citizens have the right to legally own property.",
    "Untouchability is strictly banned in Nepal.",
    "Employment rights are guaranteed to all citizens.",
    "The President acts as Nepal’s Head of State.",
    "The Prime Minister serves as the Head of Government.",
    "Following the Constitution and laws is mandatory for all citizens.",
    "Nepal’s government is divided into three administrative levels.",
    "Both state institutions and citizens are responsible for Constitution protection.",
    "Citizens have the right to request information from government bodies.",
    "Privacy is recognized as one of the fundamental rights.",
    "Children possess the right to education.",
    "Nepali women can confer citizenship to their children.",
    "Nepal functions as a federal democratic republic."
    "That’s beyond my knowledge base.",
    "I am sorry, That’s beyond my knowledge base."
]

# ---------- Nepali (देवनागरी) ----------
questions_ne = [
    "नेपालको संविधान के हो?",
    "नेपालको संविधान कहिले जारी भयो?",
    "नेपालमा सार्वभौमसत्ता कसको हुन्छ?",
    "नेपाललाई के घोषणा गरिएको छ?",
    "नेपालको राष्ट्रभाषा के हो?",
    "नागरिकलाई नागरिकताबाट वञ्चित गर्न सकिन्छ?",
    "समानताको अधिकार भन्नाले के बुझिन्छ?",
    "स्वतन्त्रताको अधिकार भन्नाले के बुझिन्छ?",
    "संविधानले मृत्युदण्डलाई अनुमति दिन्छ?",
    "नेपालको राष्ट्रिय झण्डा कस्तो छ?",
    "नेपालको राष्ट्रिय फूल के हो?",
    "नेपालको राष्ट्रिय चरा के हो?",
    "नेपालको राष्ट्रिय जनावर के हो?",
    "प्रत्येक व्यक्तिलाई के अधिकार छ?",
    "नागरिकलाई स्वास्थ्य सम्बन्धी के अधिकार छ?",
    "नेपालमा शिक्षा निःशुल्क छ?",
    "नेपालमा मतदान उमेर कति हो?",
    "नागरिकलाई सम्पत्ति अधिकार छ?",
    "नेपालमा छुवाछूत अनुमति छ?",
    "नागरिकलाई रोजगारी अधिकार छ?",
    "नेपालको राष्ट्रप्रमुख को हुन्?",
    "नेपालको सरकार प्रमुख को हुन्?",
    "प्रत्येक नागरिकले के पालना गर्नुपर्छ?",
    "नेपालमा कति तहको सरकार छ?",
    "संविधानलाई कोले संरक्षण गर्छ?",
    "नागरिकले सूचना माग्न सक्छन्?",
    "गोपनीयता आधारभूत अधिकार हो?",
    "बालबालिकालाई शिक्षा अधिकार छ?",
    "महिलाले बच्चालाई नागरिकता दिन सक्छिन्?",
    "नेपाल संघीय लोकतान्त्रिक गणतन्त्र हो?",
    "के म वर्षामा नेपालको संविधानलाई छाता जस्तै प्रयोग गर्न सक्छु?",#(irrelevant/jokes)
    "के नेपालको राष्ट्रिय चराले मोमो पकाउन जान्दछ?"#(irrelevant/jokes)
]

reference_answers_ne = [
    "नेपालको संविधान देशको मूल कानुन हो।",
    "नेपालको संविधान २०१५ सेप्टेम्बर २० मा जारी भयो।",
    "नेपालमा सार्वभौमसत्ता जनतामा हुन्छ।",
    "नेपाललाई संघीय लोकतान्त्रिक गणतन्त्र घोषणा गरिएको छ।",
    "नेपालको राष्ट्रभाषा नेपाली हो।",
    "नागरिकलाई नागरिकताबाट वञ्चित गर्न सकिँदैन।",
    "समानताको अधिकारले सबैलाई कानुन अगाडि समान बनाउँछ।",
    "स्वतन्त्रताको अधिकारले मानिसलाई स्वतन्त्र बनाउँछ।",
    "संविधानले मृत्युदण्डलाई अनुमति दिदैन।",
    "नेपालको झण्डामा दुई त्रिकोण, घाम र चन्द्रमा छन्।",
    "नेपालको राष्ट्रिय फूल लालीगुराँस हो।",
    "नेपालको राष्ट्रिय चरा डाँफे हो।",
    "नेपालको राष्ट्रिय जनावर गाई हो।",
    "प्रत्येक व्यक्तिलाई सम्मानपूर्वक बाँच्ने अधिकार छ।",
    "प्रत्येक नागरिकलाई आधारभूत स्वास्थ्य सेवा पाउने अधिकार छ।",
    "शिक्षा निश्चित तहसम्म निःशुल्क छ।",
    "मतदान उमेर १८ वर्ष हो।",
    "नागरिकलाई सम्पत्ति अधिकार हुन्छ।",
    "नेपालमा छुवाछूत निषेध गरिएको छ।",
    "नागरिकलाई रोजगारी अधिकार छ।",
    "नेपालको राष्ट्रप्रमुख राष्ट्रपति हुन्।",
    "नेपालको सरकार प्रमुख प्रधानमन्त्री हुन्।",
    "नागरिकले संविधान र कानुन पालना गर्नुपर्छ।",
    "नेपालमा संघीय, प्रदेश र स्थानीय गरी तीन तह छन्।",
    "संविधानलाई नागरिक र सरकारले संरक्षण गर्छन्।",
    "नागरिकले सूचना माग्न सक्छन्।",
    "गोपनीयता नागरिकको आधारभूत अधिकार हो।",
    "प्रत्येक बालबालिकालाई शिक्षा अधिकार हुन्छ।",
    "महिलाले बच्चालाई नागरिकता दिन सक्छिन्।",
    "हो, नेपाल संघीय लोकतान्त्रिक गणतन्त्र हो।",
    "संविधानलाई छाता जस्तै प्रयोग गर्न सकिँदैन किनभने यो एउटा कानुनी कागज हो, छाता होइन।",
    "राष्ट्रिय चराले मोमो पकाउन सक्दैन किनभने त्यो जनावर हो। मोमो पकाउने काम मान्छेले गर्छ।"
]

chatbot_answers_ne = [
    "नेपालको संविधान देशको सर्वोच्च कानुनी दस्तावेज मानिन्छ।",
    "संविधान २०१५ सेप्टेम्बर २० मा कार्यान्वयनमा आएको हो।",
    "नेपालमा सार्वभौमसत्ता जनताको स्वामित्वमा रहन्छ।",
    "नेपाललाई संघीय लोकतान्त्रिक गणतन्त्रको रूपमा चिनिन्छ।",
    "नेपालको राष्ट्रभाषा नेपाली भाषा हो।",
    "कुनै पनि नागरिकलाई नागरिकताबाट वञ्चित गर्न मिल्दैन।",
    "यसले सबैलाई कानुनको अगाडि समान बनाउँछ।",
    "यसले व्यक्तिलाई स्वतन्त्र रुपमा अभिव्यक्त र कार्य गर्न दिन्छ।",
    "संविधानले मृत्युदण्डलाई अनुमति दिएको छैन।",
    "नेपालको झण्डामा दुई त्रिकोण छन्, जसमा घाम र चन्द्रमा छन्।",
    "लालीगुराँस नेपालको राष्ट्रिय फूल हो।",
    "डाँफे नेपालमा राष्ट्रिय चराको रूपमा मानिन्छ।",
    "गाई नेपालको राष्ट्रिय जनावर हो।",
    "प्रत्येक व्यक्तिलाई गरिमापूर्वक बाँच्ने अधिकार हुन्छ।",
    "नागरिकलाई आधारभूत स्वास्थ्य सेवा प्राप्त गर्ने अधिकार छ।",
    "शिक्षा निश्चित तहसम्म निःशुल्क दिइन्छ।",
    "नेपालमा मतदान उमेर १८ वर्ष पुरा भएपछि हुन्छ।",
    "प्रत्येक नागरिकलाई सम्पत्ति राख्ने अधिकार हुन्छ।",
    "नेपालमा छुवाछूतलाई कडाईका साथ निषेध गरिएको छ।",
    "प्रत्येक नागरिकलाई रोजगारीको अधिकार छ।",
    "राष्ट्रप्रमुखको रूपमा राष्ट्रपति कार्यरत छन्।",
    "प्रधानमन्त्री सरकार प्रमुखका रूपमा कार्य गर्दछन्।",
    "सबै नागरिकले संविधान र कानुनको पालना गर्नुपर्ने हुन्छ।",
    "नेपालमा तीन तहको सरकार सञ्चालनमा छन्।",
    "संविधानको रक्षा नागरिक र राज्यले गर्छन्।",
    "नागरिकले सरोकारवाला निकायसँग सूचना माग्न सक्दछन्।",
    "गोपनीयतालाई आधारभूत अधिकारको रूपमा सुनिश्चित गरिएको छ।",
    "प्रत्येक बालबालिकालाई शिक्षाको अधिकार दिइन्छ।",
    "नेपाली महिलाले आफ्ना सन्तानलाई नागरिकता दिन सक्छिन्।",
    "हो, नेपाल संघीय लोकतान्त्रिक गणतन्त्र हो।"
    "संविधान कानूनको कागज हो, यो छाता होइन।"
    "राष्ट्रिय चराले मोमो बनाउन सक्दैनन्, किनकि तिनीहरू जनावर हुन्।"
]

# ---------- Romanized Nepali ----------
questions_ro = [
    "Nepalko sambidhan ke ho?",
    "Nepalko sambidhan kahile jari bhayo?",
    "Nepalma sarvabhaumsatta kasako hunchha?",
    "Nepal lai ke ghoshana gariyeko cha?",
    "Nepalko rashtrabhasha ke ho?",
    "Nagariklai nagarikata bata banchit garna sakincha?",
    "Samanatako adhikar bhannale ke bujhcha?",
    "Swatantratako adhikar bhannale ke bujhcha?",
    "Sambidhan le mrityudanda anumati dincha?",
    "Nepalko rashtriya jhanda kasto cha?",
    "Nepalko rashtriya phool ke ho?",
    "Nepalko rashtriya chara ke ho?",
    "Nepalko rashtriya janawar ke ho?",
    "Pratyek byakti lai ke adhikar cha?",
    "Nagarik lai swasthya sambandhi ke adhikar cha?",
    "Nepalma shiksha nishulka cha?",
    "Nepalma matadan umer kati ho?",
    "Nagariklai sampatti adhikar cha?",
    "Nepalma chhuwa chhut anumati cha?",
    "Nagariklai rojgari adhikar cha?",
    "Nepalko rashtrapramukh ko hun?",
    "Nepalko sarkar pramukh ko hun?",
    "Harek nagarikle ke palana garnu parchha?",
    "Nepalma kati tah ko sarkar cha?",
    "Sambidhanlai ko le sanrakshan garcha?",
    "Nagarikle suchana magn sakcha?",
    "Gopaniyata aadharbhut adhikar ho?",
    "Balbalikalai shiksha adhikar cha?",
    "Mahila le bachchha lai nagarikata din sakchin?",
    "Nepal sanghiya loktantrik ganatantra ho?",
    "Ke ma barshaamaa Nepalko samvidhaanlai chhaataa jastai prayog garn sakchu?",#(irrelevant/jokes)
    "Ke Nepalko raastriya charale momo pakaun jandachha?"#(irrelevant/jokes)
]

reference_answers_ro = [
    "Nepalko sambidhan deshko mul kanun ho.",
    "Nepalko sambidhan 2015 September 20 ma jari bhayo.",
    "Nepalma sarvabhaumsatta janatama hunchha.",
    "Nepal lai sanghiya loktantrik ganatantra ghoshana gariyeko cha.",
    "Nepalko rashtrabhasha Nepali ho.",
    "Nagariklai nagarikata bata banchit garna sakindaina.",
    "Samanatako adhikar sabailai kanun agadi saman banaucha.",
    "Swatantratako adhikar manis lai swatantra banaucha.",
    "Sambidhan le mrityudanda anumati didaina.",
    "Nepalko jhanda ma dui trikona, gham ra chandramā chan.",
    "Nepalko rashtriya phool Laligurans ho.",
    "Nepalko rashtriya chara Danphe ho.",
    "Nepalko rashtriya janawar gai ho.",
    "Pratyek byakti lai sammapurvak bachne adhikar cha.",
    "Pratyek nagarik lai aadharbhut swasthya sewa paune adhikar cha.",
    "Shiksha nischit tah samma nishulka cha.",
    "Matadan umer 18 barsa ho.",
    "Nagarik lai sampatti adhikar hunchha.",
    "Nepalma chhuwa chhut nished gariyeko cha.",
    "Nagarik lai rojgari adhikar hunchha.",
    "Nepalko rashtrapramukh rashtrapati hun.",
    "Nepalko sarkar pramukh pradhanmantri hun.",
    "Nagarikle sambidhan ra kanun palana garnu parchha.",
    "Nepalma sanghiya, pradesh ra sthaniya gari tin tah chan.",
    "Sambidhanlai nagarik ra sarkarle sanrakshan garchhan.",
    "Nagarikle suchana magn sakchan.",
    "Gopaniyata nagarikko aadharbhut adhikar ho.",
    "Pratyek balbalika lai shiksha adhikar hunchha.",
    "Mahila le bachchha lai nagarikata din sakchin.",
    "Ho, Nepal sanghiya loktantrik ganatantra ho.",
    "Samvidhaan kaanoonko kaagaj ho, chhaataa hoina",
    "Raastriya charale momo pakaun sakdaina"
]

chatbot_answers_ro = [
    "Nepalko sambidhan deshko sarboccha kanuni dastavej manincha.",
    "Sambidhan 2015 September 20 dekhi prayogma aayeko ho.",
    "Nepalma sarvabhaumsatta janatako haat ma hunchha.",
    "Nepal lai sanghiya loktantrik ganatantra ko roopma manincha.",
    "Nepali bhasa Nepalko rashtrabhasha ho.",
    "Kunai nagariklai nagarikata bata banchit garna milena.",
    "Yesle sabailai kanun agadi saman adhikar dincha.",
    "Yesle manis haru lai bolne ra garne swatantrata dincha.",
    "Sambidhanle mrityudanda ko anumati dindaina.",
    "Nepalko jhanda ma dui trikona cha, gham ra chandramā samet chan.",
    "Nepalko rashtriya phool ho Laligurans.",
    "Danphe Nepalma rashtriya chara ho.",
    "Gai Nepalko rashtriya janawar ho.",
    "Pratyek byakti lai maryadapurna jeevan jine adhikar hunchha.",
    "Nagarikle mool swasthya sewa paune adhikar hunchha.",
    "Nischit tah samma shiksha nishulka uplabdha hunchha.",
    "Matadan umer Nepalma 18 barsa ho.",
    "Sampatti adhikar sabai nagariklai cha.",
    "Nepalma chhuwa chhut lai pura roopma nishedh garincha.",
    "Nagarik sanga rojgari paune adhikar hunchha.",
    "Rashtrapramukh ko roopma rashtrapati hun.",
    "Sarkar pramukh bhaneko pradhanmantri hun.",
    "Sambidhan ra kanunko palana harek nagarikle garnu parchha.",
    "Nepalma tin tah ko sarkar cha: sanghiya, pradesh ra sthaniya.",
    "Sambidhan ko raksha nagarik ra sarkarle garchhan.",
    "Ho, nagarikle suchana prapt garna sakchan.",
    "Ho, gopaniyata ek aadharbhut adhikar ho.",
    "Shiksha adhikar pratyek balbalikalai cha.",
    "Ho, mahila le aafno bachchha lai nagarikata din sakchin.",
    "Ho, Nepal ek sanghiya loktantrik ganatantra ho."
    "Samvidhaanlai chhaataa jastai prayog garn sakinna.",
    "Raastriya charale momo pakaun sakdainan."
    
]

# Ground truth: 1 = correct answer, 0 = incorrect/irrelevant answer
ground_truth_en = [
    1, 1, 1, 1, 0,
    1, 0, 1, 1, 1,
    1, 1, 1, 1, 1,
    1, 1, 0, 1, 1,
    1, 1, 1, 1, 1,
    0, 1, 1, 1, 1,
    1, 1  # Last two questions are irrelevant/jokes
]

# Ground truth: 1 = correct answer, 0 = incorrect/irrelevant answer
ground_truth_ne = [
    1, 1, 0, 1, 1,
    1, 1, 1, 1, 1,
    1, 0, 1, 0, 1,
    1, 1, 1, 1, 1,
    0, 1, 1, 1, 1,
    1, 1, 1, 1, 1,
    1, 1  # Last two questions are irrelevant/jokes
]

# Ground truth: 1 = correct answer, 0 = incorrect/irrelevant answer
ground_truth_ro = [
    1, 1, 1, 1, 1,
    1, 1, 1, 1, 0,
    0, 1, 1, 1, 1,
    1, 1, 1, 1, 1,
    1, 1, 1, 0, 1,
    0, 1, 1, 1, 1,
    1, 1  # Last two questions are irrelevant/jokes
]
# --------------------------------------------------------
# Step 2: Evaluation Function
# --------------------------------------------------------

def evaluate_semantic(lang, questions, reference_answers, chatbot_answers, ground_truth, model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"):
    model = SentenceTransformer(model_name)
    threshold = 0.79
    y_true, y_pred = [], []
    semantic_scores, bleu_scores, exact_match_scores = [], [], []

    for q, ref, hyp, gt in zip(questions, reference_answers, chatbot_answers, ground_truth):
        ref_emb = model.encode(ref, convert_to_tensor=True)
        hyp_emb = model.encode(hyp, convert_to_tensor=True)
        sim_score = util.pytorch_cos_sim(ref_emb, hyp_emb).item()
        semantic_scores.append(sim_score)

        y_true.append(gt)
        y_pred.append(1 if sim_score >= threshold else 0)

        bleu = sentence_bleu([ref.split()], hyp.split(), smoothing_function=SmoothingFunction().method1)
        bleu_scores.append(bleu)

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    avg_similarity = np.mean(semantic_scores)
    avg_bleu = np.mean(bleu_scores)

    print(f"=== Performance Evaluation ({lang}) ===")
    print(f"Average Cosine Similarity: {avg_similarity:.2f}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"Average BLEU Score: {avg_bleu:.2f}")
    print("\n")
# --------------------------------------------------------
# Step 3: Run Evaluations for All Languages
# --------------------------------------------------------
evaluate_semantic("English_GPT-3.5", questions_en, reference_answers_en, chatbot_answers_en, ground_truth_en)
evaluate_semantic("Nepali_GPT-3.5", questions_ne, reference_answers_ne, chatbot_answers_ne, ground_truth_ne)
evaluate_semantic("Romanized Nepali_GPT-3.5", questions_ro, reference_answers_ro, chatbot_answers_ro, ground_truth_ro)


=== Performance Evaluation (English_GPT-3.5) ===
Average Cosine Similarity: 0.85
Accuracy: 0.70
Precision: 0.90
Recall: 0.73
F1 Score: 0.81
Average BLEU Score: 0.14


=== Performance Evaluation (Nepali_GPT-3.5) ===
Average Cosine Similarity: 0.84
Accuracy: 0.70
Precision: 0.90
Recall: 0.73
F1 Score: 0.81
Average BLEU Score: 0.20


=== Performance Evaluation (Romanized Nepali_GPT-3.5) ===
Average Cosine Similarity: 0.81
Accuracy: 0.58
Precision: 0.85
Recall: 0.63
F1 Score: 0.72
Average BLEU Score: 0.18




In [ ]:
# ================================
# Performance Evaluation Evaluation for Constitution Chatbot (BASE MODEL BERT)
# ================================

from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np

# ---------- English ----------
questions_en = [
    # 1 - 5
    "What is the Constitution of Nepal?",
    "When was the Constitution of Nepal promulgated?",
    "Who holds sovereignty in Nepal?",
    "What is Nepal declared as?",
    "What is the national language of Nepal?",
    # 6 - 10
    "Can a citizen be deprived of citizenship?",
    "What is the right to equality?",
    "What is the right to freedom?",
    "Does the Constitution allow death penalty?",
    "What is the national flag of Nepal like?",
    # 11 - 15
    "What is the national flower of Nepal?",
    "What is the national bird of Nepal?",
    "What is the national animal of Nepal?",
    "What right does every person have?",
    "What health right do citizens have?",
    # 16 - 20
    "Is education free in Nepal?",
    "What is the voting age in Nepal?",
    "Do citizens have the right to property?",
    "Is untouchability allowed in Nepal?",
    "Do citizens have the right to employment?",
    # 21 - 25
    "Who is the Head of State of Nepal?",
    "Who is the Head of Government of Nepal?",
    "What must every citizen follow?",
    "How many levels of government does Nepal have?",
    "Who protects the Constitution?",
    # 26 - 30
    "Can citizens demand information?",
    "Is privacy a fundamental right?",
    "Do children have the right to education?",
    "Can a Nepali woman give citizenship to her child?",
    "Is Nepal a federal democratic republic?"
    # 31 - 32 (irrelevant/jokes)
    "Can I use the Constitution of Nepal as an umbrella during rain?"
    "Does the national bird of Nepal know how to cook momo?"
]

reference_answers_en = [
    "The Constitution of Nepal is the basic law of the country.",
    "The Constitution was promulgated on 20 September 2015.",
    "Sovereignty in Nepal belongs to the people.",
    "Nepal is a federal democratic republic.",
    "Nepali is the national language.",
    "Citizens cannot be deprived of citizenship.",
    "The right to equality means all are equal before the law.",
    "The right to freedom means people can speak and act freely.",
    "The Constitution does not allow death penalty.",
    "Nepal’s flag has two triangles with sun and moon.",
    "The national flower is rhododendron.",
    "The national bird is Danphe.",
    "The national animal is cow.",
    "Every person has the right to live with dignity.",
    "Citizens have the right to basic health care.",
    "Education is free up to a certain level.",
    "The voting age in Nepal is 18 years.",
    "Citizens have the right to property.",
    "Untouchability is prohibited in Nepal.",
    "Citizens have the right to employment.",
    "The President is the Head of State.",
    "The Prime Minister is the Head of Government.",
    "Citizens must follow the Constitution and laws.",
    "Nepal has three levels of government.",
    "The Constitution is protected by people and state.",
    "Citizens can demand information from authorities.",
    "Privacy is a fundamental right.",
    "Children have the right to education.",
    "Nepali women have the right to confer citizenship on their children.",
    "Nepal is a federal democratic republic."
    "No, the Constitution is not waterproof.",
    "This answer is not related to the Constitution."
]

chatbot_answers_en = [
    "The Constitution of Nepal serves as the highest legal document of the country.",
    "It came into effect on 20th September 2015.",
    "In Nepal, sovereignty is vested in the people.",
    "Nepal operates as a federal democratic republic.",
    "Nepali is the official language of the nation.",
    "Citizens cannot be stripped of their citizenship.",
    "The Constitution ensures equal treatment for all before the law.",
    "It guarantees the freedom of speech and action to its citizens.",
    "No, the death penalty is prohibited in Nepal.",
    "The flag consists of two triangles, featuring the sun and the moon.",
    "The rhododendron is recognized as Nepal’s national flower.",
    "The Danphe, or Himalayan Monal, is Nepal’s national bird.",
    "The cow holds the status of the national animal of Nepal.",
    "Every person has the right to live with dignity and respect.",
    "Basic healthcare services are available to all citizens.",
    "Yes, education up to a certain level is provided for free.",
    "In Nepal, the legal age for voting is 18 years old.",
    "Every citizen has the right to own property.",
    "No, untouchability is strictly prohibited in Nepal.",
    "Every citizen has the right to work and seek employment.",
    "Nepal’s Head of State is the President.",
    "The Prime Minister is the Head of Government.",
    "It is mandatory for every citizen to abide by the Constitution and the laws.",
    "Nepal follows a three-tiered governance system: federal, provincial, and local.",
    "Both the citizens and the state share the responsibility of safeguarding the Constitution.",
    "Yes, citizens have the right to request information from government authorities.",
    "Yes, privacy is an essential right guaranteed to every citizen.",
    "Every child has the right to receive an education.",
    "A mother is empowered to grant citizenship to her child.",
    "Yes, Nepal is a federal democratic republic.",
    "I’m sorry, I don’t have information on that topic.",
    "Unfortunately, that’s beyond my knowledge at the moment."
]


# ---------- Nepali (देवनागरी) ----------
questions_ne = [
    "नेपालको संविधान के हो?",
    "नेपालको संविधान कहिले जारी भयो?",
    "नेपालमा सार्वभौमसत्ता कसको हुन्छ?",
    "नेपाललाई के घोषणा गरिएको छ?",
    "नेपालको राष्ट्रभाषा के हो?",
    "नागरिकलाई नागरिकताबाट वञ्चित गर्न सकिन्छ?",
    "समानताको अधिकार भन्नाले के बुझिन्छ?",
    "स्वतन्त्रताको अधिकार भन्नाले के बुझिन्छ?",
    "संविधानले मृत्युदण्डलाई अनुमति दिन्छ?",
    "नेपालको राष्ट्रिय झण्डा कस्तो छ?",
    "नेपालको राष्ट्रिय फूल के हो?",
    "नेपालको राष्ट्रिय चरा के हो?",
    "नेपालको राष्ट्रिय जनावर के हो?",
    "प्रत्येक व्यक्तिलाई के अधिकार छ?",
    "नागरिकलाई स्वास्थ्य सम्बन्धी के अधिकार छ?",
    "नेपालमा शिक्षा निःशुल्क छ?",
    "नेपालमा मतदान उमेर कति हो?",
    "नागरिकलाई सम्पत्ति अधिकार छ?",
    "नेपालमा छुवाछूत अनुमति छ?",
    "नागरिकलाई रोजगारी अधिकार छ?",
    "नेपालको राष्ट्रप्रमुख को हुन्?",
    "नेपालको सरकार प्रमुख को हुन्?",
    "प्रत्येक नागरिकले के पालना गर्नुपर्छ?",
    "नेपालमा कति तहको सरकार छ?",
    "संविधानलाई कोले संरक्षण गर्छ?",
    "नागरिकले सूचना माग्न सक्छन्?",
    "गोपनीयता आधारभूत अधिकार हो?",
    "बालबालिकालाई शिक्षा अधिकार छ?",
    "महिलाले बच्चालाई नागरिकता दिन सक्छिन्?",
    "नेपाल संघीय लोकतान्त्रिक गणतन्त्र हो?",
    "के म वर्षामा नेपालको संविधानलाई छाता जस्तै प्रयोग गर्न सक्छु?",#(irrelevant/jokes)
    "के नेपालको राष्ट्रिय चराले मोमो पकाउन जान्दछ?"#(irrelevant/jokes)
]

reference_answers_ne = [
    "नेपालको संविधान देशको मूल कानुन हो।",
    "नेपालको संविधान २०१५ सेप्टेम्बर २० मा जारी भयो।",
    "नेपालमा सार्वभौमसत्ता जनतामा हुन्छ।",
    "नेपाललाई संघीय लोकतान्त्रिक गणतन्त्र घोषणा गरिएको छ।",
    "नेपालको राष्ट्रभाषा नेपाली हो।",
    "नागरिकलाई नागरिकताबाट वञ्चित गर्न सकिँदैन।",
    "समानताको अधिकारले सबैलाई कानुन अगाडि समान बनाउँछ।",
    "स्वतन्त्रताको अधिकारले मानिसलाई स्वतन्त्र बनाउँछ।",
    "संविधानले मृत्युदण्डलाई अनुमति दिदैन।",
    "नेपालको झण्डामा दुई त्रिकोण, घाम र चन्द्रमा छन्।",
    "नेपालको राष्ट्रिय फूल लालीगुराँस हो।",
    "नेपालको राष्ट्रिय चरा डाँफे हो।",
    "नेपालको राष्ट्रिय जनावर गाई हो।",
    "प्रत्येक व्यक्तिलाई सम्मानपूर्वक बाँच्ने अधिकार छ।",
    "प्रत्येक नागरिकलाई आधारभूत स्वास्थ्य सेवा पाउने अधिकार छ।",
    "शिक्षा निश्चित तहसम्म निःशुल्क छ।",
    "मतदान उमेर १८ वर्ष हो।",
    "नागरिकलाई सम्पत्ति अधिकार हुन्छ।",
    "नेपालमा छुवाछूत निषेध गरिएको छ।",
    "नागरिकलाई रोजगारी अधिकार छ।",
    "नेपालको राष्ट्रप्रमुख राष्ट्रपति हुन्।",
    "नेपालको सरकार प्रमुख प्रधानमन्त्री हुन्।",
    "नागरिकले संविधान र कानुन पालना गर्नुपर्छ।",
    "नेपालमा संघीय, प्रदेश र स्थानीय गरी तीन तह छन्।",
    "संविधानलाई नागरिक र सरकारले संरक्षण गर्छन्।",
    "नागरिकले सूचना माग्न सक्छन्।",
    "गोपनीयता नागरिकको आधारभूत अधिकार हो।",
    "प्रत्येक बालबालिकालाई शिक्षा अधिकार हुन्छ।",
    "महिलाले बच्चालाई नागरिकता दिन सक्छिन्।",
    "हो, नेपाल संघीय लोकतान्त्रिक गणतन्त्र हो।",
    "संविधानलाई छाता जस्तै प्रयोग गर्न सकिँदैन किनभने यो एउटा कानुनी कागज हो, छाता होइन।",
    "राष्ट्रिय चराले मोमो पकाउन सक्दैन किनभने त्यो जनावर हो। मोमो पकाउने काम मान्छेले गर्छ।"
]

chatbot_answers_ne = [ 
    "नेपालको संविधान हाम्रो देशको सबैभन्दा ठूलो कानुनी दस्तावेज हो।",
    "नेपालको संविधान २०१५ को सेप्टेम्बर २० मा लागू भएको हो।",
    "नेपालमा सबै शक्ति जनतामा निहित छ।",
    "नेपालले संघीय लोकतान्त्रिक गणतन्त्रको घोषण गरेको छ।",
    "नेपालको राष्ट्रिय भाषा नेपाली हो।",
    "नागरिकलाई नागरिकता बाट वञ्चित गर्नु असम्भव छ।",
    "संविधानले सबैलाई कानुनको अगाडि समान बनाउँछ।",
    "यो संविधानले व्यक्तिलाई स्वतन्त्रता र अधिकार दिन्छ, जस्तै बोल्न र कार्य गर्न।",
    "संविधानले मृत्युदण्डलाई स्वीकृति दिने छैन।",
    "नेपालको झण्डामा दुई रातो त्रिकोण र त्यसमा घाम र चन्द्रमा रहेको छ।",
    "नेपालको राष्ट्रिय फूल लालीगुराँस हो।",
    "नेपालको राष्ट्रिय चरा डाँफे हो।",
    "नेपालको राष्ट्रिय जनावर गाई हो।",
    "प्रत्येक व्यक्तिलाई सम्मान र सुरक्षा प्राप्त गर्ने अधिकार छ।",
    "नागरिकलाई निःशुल्क स्वास्थ्य सेवा प्राप्त गर्ने अधिकार छ।",
    "नागरिकलाई एक निश्चित तहसम्म निःशुल्क शिक्षा प्राप्त हुन्छ।",
    "नेपालमा मतदानको उमेर १८ वर्षको छ।",
    "नागरिकलाई सम्पत्ति अधिकार कानूनी रूपमा सुरक्षित गरिएको छ।",
    "नेपालमा छुवाछूतको प्रथा विधिक रूपमा निषेध गरिएको छ।",
    "नागरिकलाई रोजगारीको अधिकार छ।",
    "नेपालको राष्ट्रप्रमुख राष्ट्रपति हुनुहुन्छ।",
    "नेपालको सरकार प्रमुख प्रधामन्त्री हुनुहुन्छ।",
    "हरेक नागरिकले संविधान र कानुनको सम्मान गर्नुपर्छ।",
    "नेपालमा संघीय, प्रदेश र स्थानीय तहको संरचना छ।",
    "संविधानलाई नागरिक र सरकारले संरक्षण गर्छन्।",
    "हो, नागरिकले सम्बन्धित निकायसँग सूचना माग्न सक्नेछन्।",
    "हो, गोपनीयता हाम्रो एक आधारभूत अधिकार हो।",
    "हो, प्रत्येक बालबालिकालाई शिक्षा प्राप्त गर्ने अधिकार छ।",
    "हो, आमाले बच्चालाई नागरिकता दिन सक्नुहुन्छ।",
    "हो, नेपाल एक संघीय लोकतान्त्रिक गणतन्त्र हो।",
    "संविधान छाता होइन, यो कानुनी कागज हो।",
    "राष्ट्रिय चराले मोमो पकाउन सक्दैनन्, किनकि उनीहरू जनावर हुन्।"
]


# ---------- Romanized Nepali ----------
questions_ro = [
    "Nepalko sambidhan ke ho?",
    "Nepalko sambidhan kahile jari bhayo?",
    "Nepalma sarvabhaumsatta kasako hunchha?",
    "Nepal lai ke ghoshana gariyeko cha?",
    "Nepalko rashtrabhasha ke ho?",
    "Nagariklai nagarikata bata banchit garna sakincha?",
    "Samanatako adhikar bhannale ke bujhcha?",
    "Swatantratako adhikar bhannale ke bujhcha?",
    "Sambidhan le mrityudanda anumati dincha?",
    "Nepalko rashtriya jhanda kasto cha?",
    "Nepalko rashtriya phool ke ho?",
    "Nepalko rashtriya chara ke ho?",
    "Nepalko rashtriya janawar ke ho?",
    "Pratyek byakti lai ke adhikar cha?",
    "Nagarik lai swasthya sambandhi ke adhikar cha?",
    "Nepalma shiksha nishulka cha?",
    "Nepalma matadan umer kati ho?",
    "Nagariklai sampatti adhikar cha?",
    "Nepalma chhuwa chhut anumati cha?",
    "Nagariklai rojgari adhikar cha?",
    "Nepalko rashtrapramukh ko hun?",
    "Nepalko sarkar pramukh ko hun?",
    "Harek nagarikle ke palana garnu parchha?",
    "Nepalma kati tah ko sarkar cha?",
    "Sambidhanlai ko le sanrakshan garcha?",
    "Nagarikle suchana magn sakcha?",
    "Gopaniyata aadharbhut adhikar ho?",
    "Balbalikalai shiksha adhikar cha?",
    "Mahila le bachchha lai nagarikata din sakchin?",
    "Nepal sanghiya loktantrik ganatantra ho?",
    "Ke ma barshaamaa Nepalko samvidhaanlai chhaataa jastai prayog garn sakchu?",#(irrelevant/jokes)
    "Ke Nepalko raastriya charale momo pakaun jandachha?"#(irrelevant/jokes)
]

reference_answers_ro = [
    "Nepalko sambidhan deshko mul kanun ho.",
    "Nepalko sambidhan 2015 September 20 ma jari bhayo.",
    "Nepalma sarvabhaumsatta janatama hunchha.",
    "Nepal lai sanghiya loktantrik ganatantra ghoshana gariyeko cha.",
    "Nepalko rashtrabhasha Nepali ho.",
    "Nagariklai nagarikata bata banchit garna sakindaina.",
    "Samanatako adhikar sabailai kanun agadi saman banaucha.",
    "Swatantratako adhikar manis lai swatantra banaucha.",
    "Sambidhan le mrityudanda anumati didaina.",
    "Nepalko jhanda ma dui trikona, gham ra chandramā chan.",
    "Nepalko rashtriya phool Laligurans ho.",
    "Nepalko rashtriya chara Danphe ho.",
    "Nepalko rashtriya janawar gai ho.",
    "Pratyek byakti lai sammapurvak bachne adhikar cha.",
    "Pratyek nagarik lai aadharbhut swasthya sewa paune adhikar cha.",
    "Shiksha nischit tah samma nishulka cha.",
    "Matadan umer 18 barsa ho.",
    "Nagarik lai sampatti adhikar hunchha.",
    "Nepalma chhuwa chhut nished gariyeko cha.",
    "Nagarik lai rojgari adhikar hunchha.",
    "Nepalko rashtrapramukh rashtrapati hun.",
    "Nepalko sarkar pramukh pradhanmantri hun.",
    "Nagarikle sambidhan ra kanun palana garnu parchha.",
    "Nepalma sanghiya, pradesh ra sthaniya gari tin tah chan.",
    "Sambidhanlai nagarik ra sarkarle sanrakshan garchhan.",
    "Nagarikle suchana magn sakchan.",
    "Gopaniyata nagarikko aadharbhut adhikar ho.",
    "Pratyek balbalika lai shiksha adhikar hunchha.",
    "Mahila le bachchha lai nagarikata din sakchin.",
    "Ho, Nepal sanghiya loktantrik ganatantra ho.",
    "Samvidhaan kaanoonko kaagaj ho, chhaataa hoina",
    "Raastriya charale momo pakaun sakdaina"
]

chatbot_answers_ro = [
    "Nepalko sambidhan deshko sabse mahan kanuni dastavej ho.",
    "Nepalko sambidhan 2015 ko September 20 tarikh ma lagu bhayo.",
    "Nepalma sarvabhaumsatta janatako ho.",
    "Nepal le sanghiya loktantrik ganatantra ko ghoshna garieko cha.",
    "Nepalko rashtrabhasha Nepali ho.",
    "Hoin, nagarik lai nagarikata bata banchit garne ko sambhavna chhaina.",
    "Yo sambidhan sabai manis lai kanun agadi saman banaucha.",
    "Yo manis lai swatantra roopma bolne ra kaam garne adhikar dincha.",
    "Hoin, sambidhan le mrityu dandako anumati diyen.",
    "Nepalko jhanda dui rato trikona cha jasma gham ra chandramā chan.",
    "Nepalko rashtriya phool Laligurans ho.",
    "Nepalko rashtriya chara Danphe ho.",
    "Nepalko rashtriya janawar gai ho.",
    "Ho, pratyek manish lai sammapurvak bachne adhikar cha.",
    "Ho, nagarik lai aadharbhut swasthya sewa nishulka roop ma prapt cha.",
    "Ho, nagarik lai nischit tah samma shiksha nishulka milcha.",
    "Nepalma matadan ko umar 18 barsa ho.",
    "Ho, nagarik lai sampatti ko adhikar sunischit cha.",
    "Hoin, Nepalma chhuwa-chhut ko pratha nishedh gariyeko cha.",
    "Ho, nagarik lai rojgar ko adhikar cha.",
    "Nepalko rashtrapramukh rashtrapati ho.",
    "Nepalko sarkar pramukh pradhanmantri ho.",
    "Harek nagarik le sambidhan ra kanun ko palana garne daayitwa cha.",
    "Nepalma sanghiya, pradesh ra sthaniya sarkar tin tah cha.",
    "Sambidhan ko sanrakshan nagarik ra sarkar le milera garchhan.",
    "Ho, nagarik le sambandhit nikay sanga suchana magn sakchan.",
    "Ho, gopaniyata ek aadharbhut adhikar ho.",
    "Ho, sabai balbalikalai shiksha ko adhikar cha.",
    "Ho, aama le bachchha lai nagarikata din sakchin.",
    "Ho, Nepal sanghiya loktantrik ganatantra ho.",
    "Samvidhaan lai chhaataa jasari prayog garna sakdaina, kinabhane yo ek kanuni kaagaj ho, chhaataa hoina.",
    "Raastriya chara momo pakaune kam garna sakdainan, kinabhane tyo janawar ho."
]


# Ground truth: 1 = correct answer, 0 = incorrect/irrelevant answer
ground_truth_en = [
    1, 1, 1, 1, 1,
    1, 0, 1, 0, 1,
    1, 1, 1, 1, 1,
    1, 1, 1, 0, 1,
    0, 1, 0, 1, 1,
    1, 1, 1, 1, 1,
    1, 1  
]

# Ground truth: 1 = correct answer, 0 = incorrect/irrelevant answer
ground_truth_ne = [
    1, 1, 1, 1, 1,
    1, 1, 0, 1, 1,
    1, 1, 1, 1, 1,
    1, 0, 0, 1, 1,
    1, 1, 1, 1, 1,
    1, 1, 1, 0, 1,
    1, 1  
]

# Ground truth: 1 = correct answer, 0 = incorrect/irrelevant answer
ground_truth_ro = [
    1, 1, 1, 1, 1,
    1, 1, 0, 1, 1,
    1, 1, 1, 1, 1,
    0, 1, 0, 1, 1,
    1, 1, 1, 1, 1,
    0, 1, 0, 1, 1,
    1, 1  
]
# --------------------------------------------------------
# Step 2: Evaluation Function
# --------------------------------------------------------

def evaluate_semantic(lang, questions, reference_answers, chatbot_answers, ground_truth, model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"):
    model = SentenceTransformer(model_name)
    threshold = 0.79
    y_true, y_pred = [], []
    semantic_scores, bleu_scores, exact_match_scores = [], [], []

    for q, ref, hyp, gt in zip(questions, reference_answers, chatbot_answers, ground_truth):
        ref_emb = model.encode(ref, convert_to_tensor=True)
        hyp_emb = model.encode(hyp, convert_to_tensor=True)
        sim_score = util.pytorch_cos_sim(ref_emb, hyp_emb).item()
        semantic_scores.append(sim_score)

        y_true.append(gt)
        y_pred.append(1 if sim_score >= threshold else 0)

        bleu = sentence_bleu([ref.split()], hyp.split(), smoothing_function=SmoothingFunction().method1)
        bleu_scores.append(bleu)

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    avg_similarity = np.mean(semantic_scores)
    avg_bleu = np.mean(bleu_scores)

    print(f"=== Performance Evaluation ({lang}) ===")
    print(f"Average Cosine Similarity: {avg_similarity:.2f}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"Average BLEU Score: {avg_bleu:.2f}")
    print("\n")

# --------------------------------------------------------
# Step 3: Run Evaluations for All Languages
# --------------------------------------------------------
evaluate_semantic("English_BERT_Base", questions_en, reference_answers_en, chatbot_answers_en, ground_truth_en)
evaluate_semantic("Nepali_BERT_Base", questions_ne, reference_answers_ne, chatbot_answers_ne, ground_truth_ne)
evaluate_semantic("Romanized Nepali_BERT_Base", questions_ro, reference_answers_ro, chatbot_answers_ro, ground_truth_ro)

=== Performance Evaluation (English_BERT_Base) ===
Average Cosine Similarity: 0.84
Accuracy: 0.70
Precision: 0.94
Recall: 0.68
F1 Score: 0.79
Average BLEU Score: 0.16


=== Performance Evaluation (Nepali_BERT_Base) ===
Average Cosine Similarity: 0.86
Accuracy: 0.66
Precision: 0.87
Recall: 0.71
F1 Score: 0.78
Average BLEU Score: 0.26


=== Performance Evaluation (Romanized Nepali_BERT_Base) ===
Average Cosine Similarity: 0.84
Accuracy: 0.66
Precision: 0.90
Recall: 0.67
F1 Score: 0.77
Average BLEU Score: 0.33


